In [2]:
import pandas as pd
filepath = 'DataFiles\Assignment2\olympics.csv'

In [12]:
df = pd.read_csv(filepath, index_col = 0,skiprows = 1)
df.index
df.columns
#Data Cleaning - Fixing Row Index , Renaming columns ,Eliminating the unwanted rows
for column in df.columns:
#     print(column[0:2])
    if column[0:2] == '01':
        df.rename(columns={column:'Gold'+column[4:]},inplace=True)
    if column[0:2] == '02':
        df.rename(columns={column:'Silver'+column[4:]},inplace=True)
    if column[:2] == '03':
        df.rename(columns={column:'Bronze'+column[4:]},inplace=True)
    if column[:1] == '№':
        df.rename(columns={column:'#'+column[1:]},inplace=True)
names = df.index.str.split('\s\(')
df.index = names.str[0]
df['ID'] = names.str[1].str[:3]
df.drop('Totals',axis = 0,inplace=True)
df.index[0]

'Afghanistan'

In [36]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

In [3]:
df.loc[['Afghanistan','United States']]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [60]:
def answer_one():
    print(df[df['Gold'] == df['Gold'].max()].index[0])
answer_one()


United States


In [64]:
m = df['Gold'].max()
c = 0
for country in df.values:
    c+=1
    if country[1] == m:
        print(df.index[c-1])


United States


### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [8]:
def answer_two():
    c = 0
    li = list(df['Gold']-df['Gold.1'])
    df['Diffrence'] = li
    m = max(li)
    for country in df.values:
        c+=1
        if country[-1] == m:
            print(df.index[c-1])
     
    
answer_two()

United States


### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [38]:
import numpy as np
df['Diff1'] = np.logical_and(df['Gold']>0,df['Gold.1']>0)
df['GoldDif'] = np.where(df['Diff1'] == True,(df['Gold']-df['Gold.1'])/df['Gold.2'],0)
df[df['GoldDif'] == df['GoldDif'].max()].index[0]

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [10]:
def answer_four():
    df["Points"] = df["Gold.2"]*3 + df["Silver.2"]*2 + df["Bronze.2"]*1
    return df["Points"]
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [3]:
cdf = pd.read_csv('DataFiles\Assignment2\census.csv')
# li = []
# d = {}

# for i in cdf['STNAME']:
#     if i not in li:
#         li.append(i)
# print(li)
# for i in cdf['STNAME']:
#     s = 0
#     if i in li and i not in d.keys():
#         d[i] = cdf['COUNTY']
li = list(cdf['COUNTY'])
l1 = list(cdf['STNAME'])
d = {}
for i in range(len(li)):
    if l1[i] in d:
        d[l1[i]]+=li[i]
    else:
        d[l1[i]] = li[i]
m = max(d.values())
m
for i in d.items():
    if i[1] == m:
        print(i[0])
        

Texas


### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [6]:
def answer_six():
    df=cdf[cdf['SUMLEV']==50].copy()
    top3_df=pd.DataFrame(columns=['CENSUS'])
    for group,frame in df.groupby(['STNAME']):
        total=frame.sort_values('CENSUS2010POP',ascending=False)[:3]['CENSUS2010POP'].sum()
        top3_df.loc[group]=total
    return list(top3_df.sort_values('CENSUS',ascending=False)[:3].index)
answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*